# DHS Data preparation

This notebook is used to prepare the DHS data.

In [1]:
import pandas as pd
import requests
from io import BytesIO
from gzip import GzipFile

/home/elwazi_mru/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 1.0 Download the DHS Vocabulary metadata and read into a dataframe

In [2]:
# Define URLs and paths for the metadadata
dhs_index_vocab_metadata_url = "http://www.meuleman.org/DHS_Index_and_Vocabulary_metadata.tsv"
dhs_index_vocab_metadata_path = "./Problem2_DHS_data/DHS_Index_and_Vocabulary_metadata.tsv"

In [3]:
# Download the tsv file
response = requests.get(dhs_index_vocab_metadata_url)
with open(dhs_index_vocab_metadata_path, 'wb') as file:
    file.write(response.content)

In [4]:
# Read the DHS vocabulary metadata into a dataframe
data_df_vocab = pd.read_csv(dhs_index_vocab_metadata_path, sep="\t")
data_df_vocab

,library order,Biosample name,Vocabulary representative,DCC Experiment ID,DCC Library ID,DCC Biosample ID,DCC File ID,Altius Aggregation ID,Altius Library ID,Altius Biosample ID,...,Library cleanup,DNaseI units/mL,Amount Nucleic Acid (ng),Nuclei count,Protease inhibitor,Library sequencing date,Reads used,DCC SPOT score,Per-biosample peaks,DHSs in Index
0,1.0,GM06990,NaN,ENCSR000EMQ,ENCLB435ZZZ,ENCBS057ENC,ENCFF983CTQ,AG5636,LN1203,DS7748,...,Sucrose,NaN,50,NaN,NaN,2009-02-23,142681590.0,0.6790,83639.0,82918.0
1,2.0,HepG2,NaN,ENCSR000ENP,ENCLB480ZZZ,ENCBS114ENC,ENCFF419JVG,AG5635,LN1207,DS7764,...,Sucrose,NaN,50,NaN,NaN,2009-02-23,138826342.0,0.5858,89748.0,89235.0
2,3.0,hTH1,NaN,ENCSR000EQC,ENCLB591ZZZ,ENCBS345AAA,ENCFF575KOF,AG5634,LN1222,DS7840,...,Sucrose,6.0,534.9,NaN,NaN,2007-06-06,149158633.0,0.6470,94360.0,93665.0
3,4.0,Hela,NaN,ENCSR000ENO,ENCLB479ZZZ,ENCBS890POO,ENCFF503PAE,AG4219,LN1264,DS8200,...,new Sucrose,4.0,50,NaN,NaN,2007-08-24,23372724.0,0.6444,59098.0,59024.0
4,5.0,CACO2,NaN,ENCSR000EMI,ENCLB422ZZZ,ENCBS391ENC,ENCFF977BRD,AG4218,LN1269,DS8235,...,Sucrose,8.0,1,NaN,NaN,2007-09-05,22760059.0,0.7190,29894.0,29724.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,730.0,fBone_femur,Musculoskeletal,ENCSR805XIF,ENCLB236BWV,ENCBS337FPV,ENCFF604WIO,AG7442,LN45038B,DS36206B,...,NaN,NaN,8.8,1050000.0,A+Sucrose,2017-02-17,252066174.0,0.5823,146918.0,145356.0
730,731.0,fLiver,NaN,ENCSR562FNN,ENCLB638FEH,ENCBS275VNY,ENCFF795ZXN,AG7443,LN45070C,DS37372C,...,NaN,NaN,4.48,2140000.0,A+Sucrose,NaN,190541422.0,0.3703,76639.0,75369.0
731,732.0,fPlacenta,NaN,ENCSR552RKI,ENCLB423VBC,ENCBS565KNL,ENCFF084UVH,AG8805,LN45072C,DS37386C,...,NaN,NaN,1.325,1050000.0,A+Sucrose,NaN,203699532.0,0.3869,107611.0,106022.0
732,733.0,fPlacenta,Placental / trophoblast,ENCSR552XJI,ENCLB711ZZZ,ENCBS723HLT,ENCFF593AWN,AG7450,LN45076C,DS37716C,...,NaN,NaN,0.972,1380000.0,A+Sucrose,NaN,206456483.0,0.4356,115898.0,114344.0


## 2.0 Download the DHS Index and Vocabulary data and read into a dataframe

In [5]:
# Define URLs and paths for the dadata
dhs_index_vocab_url = "http://www.meuleman.org/DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz"
dhs_index_vocab_path = "./Problem2_DHS_data/DHS_Index_and_Vocabulary_hg38_WM20190703.gz"

In [6]:
# Download the gz file
response = requests.get(dhs_index_vocab_url)
with open(dhs_index_vocab_path, 'wb') as file:
    file.write(response.content)

In [22]:
# Read the compressed tab-limited txt file directly into a dataframe
with GzipFile(dhs_index_vocab_path) as gzfile:
    data_df_index_Vocab = pd.read_csv(gzfile, sep="\t")

/tmp/ipykernel_82210/3105284989.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df_index_Vocab = pd.read_csv(gzfile, sep="\t")


# 3.0 Filter out rows to retain most significant DHSs.
## Using Z-Scores
We will use a statistical approach, Z-scores, which measure the number of standard deviations an element is from the mean. This method is particularly useful if we are interested in finding DHSs that are significantly higher (or lower) in numsamples and mean_signal relative to the overall distribution:

In [23]:
# Calculate Z-scores for 'numsamples' and 'mean_signal'
data_df_index_Vocab['numsamples_z'] = (data_df_index_Vocab['numsamples'] - data_df_index_Vocab['numsamples'].mean()) / data_df_index_Vocab['numsamples'].std()
data_df_index_Vocab['mean_signal_z'] = (data_df_index_Vocab['mean_signal'] - data_df_index_Vocab['mean_signal'].mean()) / data_df_index_Vocab['mean_signal'].std()

# Filter based on Z-scores (e.g., keep entries with Z-scores above 1)
data_df_index_Vocab_short = data_df_index_Vocab[
    (data_df_index_Vocab['numsamples_z'] > 1) &
    (data_df_index_Vocab['mean_signal_z'] > 1)
]

In [24]:
data_df_index_Vocab_short

,seqname,start,end,identifier,mean_signal,numsamples,summit,core_start,core_end,component,numsamples_z,mean_signal_z
20,chr1,181400,181564,1.100655,1.712263,540,181490,181450.0,181490.0,Tissue invariant,7.762808,1.141472
45,chr1,629160,629310,1.102262,40.454979,281,629230,629210.0,629270.0,Stromal A,3.887120,42.390107
47,chr1,629520,629596,1.102264,5.152820,187,629590,629530.0,629590.0,Stromal A,2.480500,4.804567
48,chr1,629870,630020,1.102264,146.867423,730,629930,629930.0,629970.0,Tissue invariant,10.605976,155.685418
50,chr1,630181,630319,1.102266,41.684094,143,630270,630210.0,630310.0,Stromal A,1.822082,43.698722
...,...,...,...,...,...,...,...,...,...,...,...,...
3590512,chrY,19354160,19354375,Y.40428,1.702447,101,19354270,19354230.0,19354300.0,Stromal B,1.193592,1.131021
3590609,chrY,19567050,19567360,Y.407629,4.718289,395,19567210,19567170.0,19567293.0,Tissue invariant,5.593021,4.341931
3590719,chrY,19744660,19745060,Y.41039,2.006189,379,19744810,19744730.0,19744990.0,Tissue invariant,5.353596,1.454409
3590982,chrY,20575532,20575800,Y.423413,5.157244,396,20575670,20575630.0,20575710.0,Tissue invariant,5.607985,4.809277


# 4.0 Define Cardiac Tissue-Specific DHSs
First, define "heart-related" versus "not-heart-related" 
Heart-Related: Sequences derived from tissues such as cardiomyocytes, cardiac fibroblasts, endothelial cells from heart tissues, etc.
Not-Heart-Related: Sequences derived from all other non-cardiac tissues.

In [25]:
# Filter heart-related vocab
heart_related_vocab = data_df_vocab[
    (data_df_vocab['System'] == "Cardiovascular") |
    (data_df_vocab['Subsystem'].isin(["Cardiac Muscle", "Cardiac"])) |
    (data_df_vocab['Organ'] == "Heart")
]
heart_related_vocab

,library order,Biosample name,Vocabulary representative,DCC Experiment ID,DCC Library ID,DCC Biosample ID,DCC File ID,Altius Aggregation ID,Altius Library ID,Altius Biosample ID,...,Library cleanup,DNaseI units/mL,Amount Nucleic Acid (ng),Nuclei count,Protease inhibitor,Library sequencing date,Reads used,DCC SPOT score,Per-biosample peaks,DHSs in Index
17,18.0,HUVEC,Vascular / endothelial,ENCSR000EOQ,ENCLB533ZZZ,ENCBS112ENC,ENCFF011FHJ,AG4209,LN1812,DS10060,...,Sucrose,60.0,24.6,NaN,NaN,2008-12-10,383483914.0,0.4182,175498.0,174822.0
41,42.0,H7_hESC_T14,NaN,ENCSR000EMY,ENCLB447ZZZ,ENCBS532SEX,ENCFF575JJO,AG4189,LN2329,DS11814,...,minElute,120.0,2.2,NaN,NaN,2009-08-07,28331633.0,0.3604,53849.0,53784.0
48,49.0,H7_hESC_T5,NaN,ENCSR000EMW,ENCLB444ZZZ,ENCBS430FQD,ENCFF358BDI,AG4183,LN2375,DS11953,...,minElute,60.0,8.7,NaN,NaN,2009-08-24,27961281.0,0.3867,61948.0,61860.0
53,54.0,H7_hESC_T14,NaN,ENCSR000EMY,ENCLB448ZZZ,ENCBS294AAA,ENCFF936NEG,AG4178,LN2425,DS12147,...,minElute,120.0,10.3,NaN,NaN,2009-09-02,26153486.0,0.3755,52350.0,52293.0
75,76.0,HCF,NaN,ENCSR000ENH,ENCLB464ZZZ,ENCBS756HNZ,ENCFF996CVV,AG4159,LN2540,DS12491,...,minElute,120.0,14.3,NaN,NaN,2009-09-30,27918106.0,0.6044,80246.0,80178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,688.0,fHeart,Cardiac,ENCSR749BWV,ENCLB491BID,ENCBS320XNN,ENCFF981VXY,AG4019,LN31529A,DS37473A,...,NaN,NaN,3,2400000.0,A+Sucrose,2015-06-01,218768197.0,0.4300,141622.0,141121.0
688,689.0,fLeftAtrium,Cardiac,ENCSR412NMI,ENCLB226FNM,ENCBS982OCU,ENCFF741AUL,AG6213,LN31530A,DS37460A,...,NaN,NaN,3,529000.0,A+Sucrose,2015-06-01,212960129.0,0.3496,108967.0,108488.0
691,692.0,fLeftVentricle,Cardiac,ENCSR501FWC,ENCLB699DLF,ENCBS936JZF,ENCFF957QNS,AG3860,LN31860S,DS37292S,...,NaN,NaN,1.9,2320000.0,A+Sucrose,2015-07-01,203158085.0,0.3587,134029.0,133445.0
702,703.0,fHeartFibroblasts,NaN,ENCSR856NXV,ENCLB584PEG,ENCBS385GBH,ENCFF409ZKB,AG6217,LN34648A,DS39481A,...,NaN,NaN,3.336,900000.0,PBS+A,2016-01-06,199930072.0,0.5550,200548.0,200221.0


In [27]:
# Create a binary label for heart_related
data_df_index_Vocab_short['heart_related'] = data_df_index_Vocab_short['component'].isin(heart_related_vocab['Vocabulary representative']).astype(int)

/tmp/ipykernel_82210/161916716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_index_Vocab_short['heart_related'] = data_df_index_Vocab_short['component'].isin(heart_related_vocab['Vocabulary representative']).astype(int)


In [28]:
data_df_index_Vocab_short

,seqname,start,end,identifier,mean_signal,numsamples,summit,core_start,core_end,component,numsamples_z,mean_signal_z,heart_related
20,chr1,181400,181564,1.100655,1.712263,540,181490,181450.0,181490.0,Tissue invariant,7.762808,1.141472,0
45,chr1,629160,629310,1.102262,40.454979,281,629230,629210.0,629270.0,Stromal A,3.887120,42.390107,0
47,chr1,629520,629596,1.102264,5.152820,187,629590,629530.0,629590.0,Stromal A,2.480500,4.804567,0
48,chr1,629870,630020,1.102264,146.867423,730,629930,629930.0,629970.0,Tissue invariant,10.605976,155.685418,0
50,chr1,630181,630319,1.102266,41.684094,143,630270,630210.0,630310.0,Stromal A,1.822082,43.698722,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590512,chrY,19354160,19354375,Y.40428,1.702447,101,19354270,19354230.0,19354300.0,Stromal B,1.193592,1.131021,1
3590609,chrY,19567050,19567360,Y.407629,4.718289,395,19567210,19567170.0,19567293.0,Tissue invariant,5.593021,4.341931,0
3590719,chrY,19744660,19745060,Y.41039,2.006189,379,19744810,19744730.0,19744990.0,Tissue invariant,5.353596,1.454409,0
3590982,chrY,20575532,20575800,Y.423413,5.157244,396,20575670,20575630.0,20575710.0,Tissue invariant,5.607985,4.809277,0


In [29]:
# create dhs_id
data_df_index_Vocab_short['dhs_id'] = data_df_index_Vocab_short[['seqname', 'start', 'end', 'summit']].apply(lambda x: '_'.join(map(str, x)), axis=1)

/tmp/ipykernel_82210/577006869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_index_Vocab_short['dhs_id'] = data_df_index_Vocab_short[['seqname', 'start', 'end', 'summit']].apply(lambda x: '_'.join(map(str, x)), axis=1)


In [30]:
data_df_index_Vocab_short.columns


Index(['seqname', 'start', 'end', 'identifier', 'mean_signal', 'numsamples',
       'summit', 'core_start', 'core_end', 'component', 'numsamples_z',
       'mean_signal_z', 'heart_related', 'dhs_id'],
      dtype='object')

In [31]:
data_df_index_Vocab_short=data_df_index_Vocab_short[['dhs_id','seqname', 'start', 'end', 'identifier', 'mean_signal', 'numsamples',
       'summit', 'core_start', 'core_end', 'component', 'numsamples_z',
       'mean_signal_z', 'heart_related']]
data_df_index_Vocab_short

,dhs_id,seqname,start,end,identifier,mean_signal,numsamples,summit,core_start,core_end,component,numsamples_z,mean_signal_z,heart_related
20,chr1_181400_181564_181490,chr1,181400,181564,1.100655,1.712263,540,181490,181450.0,181490.0,Tissue invariant,7.762808,1.141472,0
45,chr1_629160_629310_629230,chr1,629160,629310,1.102262,40.454979,281,629230,629210.0,629270.0,Stromal A,3.887120,42.390107,0
47,chr1_629520_629596_629590,chr1,629520,629596,1.102264,5.152820,187,629590,629530.0,629590.0,Stromal A,2.480500,4.804567,0
48,chr1_629870_630020_629930,chr1,629870,630020,1.102264,146.867423,730,629930,629930.0,629970.0,Tissue invariant,10.605976,155.685418,0
50,chr1_630181_630319_630270,chr1,630181,630319,1.102266,41.684094,143,630270,630210.0,630310.0,Stromal A,1.822082,43.698722,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590512,chrY_19354160_19354375_19354270,chrY,19354160,19354375,Y.40428,1.702447,101,19354270,19354230.0,19354300.0,Stromal B,1.193592,1.131021,1
3590609,chrY_19567050_19567360_19567210,chrY,19567050,19567360,Y.407629,4.718289,395,19567210,19567170.0,19567293.0,Tissue invariant,5.593021,4.341931,0
3590719,chrY_19744660_19745060_19744810,chrY,19744660,19745060,Y.41039,2.006189,379,19744810,19744730.0,19744990.0,Tissue invariant,5.353596,1.454409,0
3590982,chrY_20575532_20575800_20575670,chrY,20575532,20575800,Y.423413,5.157244,396,20575670,20575630.0,20575710.0,Tissue invariant,5.607985,4.809277,0


In [32]:
# Save the filtered dataframe
data_df_index_Vocab_short.to_csv("./Problem2_DHS_data/heart_related_dhs.csv", index=False)

# 5.0 Extract the DHS sequences

In [33]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd

# Assuming data_df_index_Vocab_short is your DataFrame with 'seqname', 'start', 'end', and 'heart_related' columns
# Load the FASTA file
fasta_file = "./hg38.fa"  # Replace with your FASTA file path
genome = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

# Function to extract sequence
def extract_sequence(row, genome):
    chrom = row['seqname']
    start = row['start']
    end = row['end']
    # The chromosome name in the FASTA file might start with 'chr', you might need to adjust this line
    seq = genome.get(chrom, None)
    if seq:
        return str(seq.seq[start-1:end])  # -1 because genomic positions are 1-based and Python is 0-based
    else:
        return None

# Apply the function to each row in the DataFrame
data_df_index_Vocab_short['sequence'] = data_df_index_Vocab_short.apply(extract_sequence, axis=1, genome=genome)

# Now, create a new DataFrame with sequences and the 'heart_related' column
sequences_df = data_df_index_Vocab_short[['dhs_id','sequence', 'heart_related']].dropna()

# Check the first few entries
print(sequences_df.head())


                       dhs_id  \
20  chr1_181400_181564_181490   
45  chr1_629160_629310_629230   
47  chr1_629520_629596_629590   
48  chr1_629870_630020_629930   
50  chr1_630181_630319_630270   

                                             sequence  heart_related  
20  cgcccaggggaggaggcgtggcgcaggcgcagagaggcgcgccgtg...              0  
45  CACAAACATTATTATAATAAACACCCTCACCACTACAATCTTCCTA...              0  
47  ccatccctgagaatccaaaattctccgtgccacctatcacacccca...              0  
48  CAATATACTCTCCGGACAATGAACCATAACCAATACCACCAATCAA...              0  
50  ACTCCTCAATTACCCACATAGGATGAATAACAGCAGTTCTACCGTA...              0  


/tmp/ipykernel_82210/177440908.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_index_Vocab_short['sequence'] = data_df_index_Vocab_short.apply(extract_sequence, axis=1, genome=genome)


In [35]:
sequences_df.to_csv("../DNABERT/examples/dhs2/heart_specific_dhs_sequences.tsv",sep='\t', index=False, header=True)

## Using DNABERT
### 1. Finetuning the DNABERT kmer 6 model
In case we want to finetune the pretrained DNABERT model, we must have the Train and Test datasets such that we can finetune the pre-trained DNABERT model using kmers=6 using the train data and test using the test dataset.

### 2. Using the finetuned pre-trained DNABERT model
In case we wish to use the pre-trained finetuned model using kmers=6, we must download it from the [drive specified in the DNABERT repo](https://drive.google.com/drive/folders/15wFcukTv3ecPw9_25dcOv-bZmj-8d_-6). That drive contains a folder named <b>dna_model</b> with 5 files. We must download the folder dna_model and save it in our folder ```../DNABERT/examples/dhs2/ft/6/```.

If using the finetuned pretrained DNABERT model, we will use the whole dataset to make predictions.

# 6.0 Prepare the data by extracting 6-mers as we will use the DNABERT model trained with k-mers=6

In [37]:
# Split into 6-mers (Python doesn't have a direct equivalent to R's sapply function)
def split_into_6mers(sequence):
    return ' '.join([sequence[i:i+6] for i in range(len(sequence) - 5)])

# Assuming you have a DataFrame 'sequences_df' with the extracted sequences
sequences_df['sequence_6mers'] = sequences_df['sequence'].apply(split_into_6mers)

In [42]:
sequences_df1=sequences_df[['dhs_id','sequence_6mers', 'heart_related']].rename(columns={'heart_related': 'label'})
# Write the DataFrame to a TSV file
sequences_df1.to_csv("../DNABERT/examples/dhs2/ft/6/heart_specific_dhs_6mers_with_dhs_ids.tsv", sep="\t", index=False)

In [44]:
# Select the 'sequence_6mers' and 'heart_related' columns, and rename 'heart_related' to 'label'
sequences_df_final = sequences_df[['sequence_6mers', 'heart_related']].rename(columns={'heart_related': 'label', 'sequence_6mers':'sequence'})
sequences_df_final

,sequence,label
20,cgccca gcccag cccagg ccaggg cagggg agggga gggg...,0
45,CACAAA ACAAAC CAAACA AAACAT AACATT ACATTA CATT...,0
47,ccatcc catccc atccct tccctg ccctga cctgag ctga...,0
48,CAATAT AATATA ATATAC TATACT ATACTC TACTCT ACTC...,0
50,ACTCCT CTCCTC TCCTCA CCTCAA CTCAAT TCAATT CAAT...,0
...,...,...
3590512,tgtgag gtgagc tgagct gagctg agctgt gctgtt ctgt...,1
3590609,CTAGAA TAGAAC AGAACG GAACGT AACGTT ACGTTG CGTT...,0
3590719,TAGCTG AGCTGC GCTGCT CTGCTT TGCTTA GCTTAC CTTA...,0
3590982,GGGCCC GGCCCC GCCCCG CCCCGC CCCGCC CCGCCC CGCC...,0


In [45]:
# Write the DataFrame to a TSV file
sequences_df_final.to_csv("../DNABERT/examples/dhs2/ft/6/heart_specific_dhs_6mers.tsv", sep="\t", index=False)

### Next we have to create the train, test datasets and save.

In [46]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
train_df, test_df = train_test_split(sequences_df_final, test_size=0.2, random_state=42)

# Save the training and testing sets to .tsv files
train_df.to_csv('../DNABERT/examples/dhs2/ft/6/heart_specific_dhs_6mers_train.tsv', sep='\t', index=False, header=False)
test_df.to_csv('../DNABERT/examples/dhs2/ft/6/heart_specific_dhs_6mers_test.tsv', sep='\t', index=False, header=False)

## Important Note
When running prediction, the data that has to be used for prediction must be named <b>dev.tsv</b> in folder:
../DNABERT/examples/dhs2/ft/6/